To obtain all the counterterms that emerge at each order in the perturbation expansion.

The different counterterm diagrams at each order will be of the form.

As the order increases the number of  

For example:
- 2nd order we would only have the counterterm 1-o-1.
- 3rd order we would have the counterterms 1-o-2 and 2-o-1.
- 4th order we would have: 1-o-1, 1-o-3, 3-o-1, 2-o-2.
- 5th order: 1-o-2, 2-o-1, 1-o-4, 4-o-1, 2-o-3, 3-o-2.
- 6th order: 1-o-1, 1-o-3, 3-o-1, 2-o-2, 1-o-5, 5-o-1, 2-o-4, 4-o-2, 3-o-3.
and so on.

So at each order we would have diagrams with the same appearance as the previous order of the same parity plus other diagrams which the sum of the number of incomming and outgoing particles equal the order considered.


In [ ]:
def return_counterterm_diagrams (order):
    